In [74]:
import textacy
import textacy.vsm
import spacy
nlp = spacy.load('en_core_web_sm') ## install it by python -m spacy download en (run as administrator)

import glob
import pandas as pd
import numpy as np
import collections
import re

In [3]:
files = glob.glob('./incoPat_225509/*.xls')

dfs = []
for f in files:
    try:
        dfs.append(pd.read_excel(f))
    except Exception as e:
        print("{0}: {1}".format(f, e))

df = pd.concat(dfs)
df = df.reset_index(drop=True)

In [4]:
docs = df['标题(英)'].apply(lambda x: nlp(x.lower())).to_list()

In [8]:
vectorizer.weighting

'tf * log((n_docs + 1) / (df + 1)) + 1'

In [9]:
corpus = textacy.Corpus(nlp, data =docs)

In [87]:
vectorizer = textacy.vsm.Vectorizer(
    tf_type="linear", apply_idf=True, idf_type="smooth", apply_dl=False, min_df=0.0001, max_df=0.01) 
doc_term_matrix = vectorizer.fit_transform(
    (doc._.to_terms_list(ngrams=2,normalize='lemma', entities=False, as_strings=True, filter_stops=True, filter_nums=True, filter_punct=True, drop_determiners=True) for doc in corpus))

In [88]:
# print(vectorizer.terms_list)
# print(doc_term_matrix.todense())

In [89]:
tfidf_dict = {}
np_matrix = doc_term_matrix.todense()
for term in vectorizer.terms_list:
    tfidf_dict[term] = np.max(np_matrix[:,vectorizer.vocabulary_terms[term]])
sorted_x = sorted(tfidf_dict.items(), key=lambda kv: kv[1], reverse=True)
sorted_dict = collections.OrderedDict(sorted_x)

In [102]:
filter_dict = collections.OrderedDict()
re_ = r"^A |^THE |^THEIR |^ITS |^THIS |^AN |^SUCH A |^AS |^TO |^AND |^-PRON- |AT LEAST|USE THEREOF|^OR | THEREOF| THEREFOR| THERETO| THEREFROM| THEREBY"
for k, v in sorted_dict.items():
    k = re.sub(re_, "", k.upper())
    if  k.count(' ') < 4 and  k.count(' ') >= 1 and k.count('(') == 0 and k.count(')') == 0:
        filter_dict[k] = v
filter_df = pd.DataFrame.from_dict(filter_dict, orient='index', columns=['score'])

In [104]:
filter_df.to_excel('./out/kws_tfidf.xlsx')

In [100]:
filter_dict

OrderedDict([('ALKALINE STORAGE', 59.54953440706481),
             ('STATE SECONDARY', 59.352795470126864),
             ('PLASTIC LENS', 57.38951800287637),
             ('ORGANIC SEMICONDUCTOR', 55.33909348497137),
             ('AQUEOUS SECONDARY', 53.30081115809585),
             ('STORAGE BATTERY', 51.23176824034547),
             ('ION SECONDARY', 51.06804165804853),
             ('SYNTHETIC POLYMER', 50.740329762458394),
             ('ACID BASE', 49.969576363322105),
             ('LAYERED PRODUCT', 49.969576363322105),
             ('SILICA PARTICLE', 49.62461200588734),
             ('SURFACE EMIT', 49.62461200588734),
             ('TRANSPARENT CONDUCTIVE', 48.798977591696485),
             ('LITHIUM ION', 48.38862577970587),
             ('IMAGE CAPTURE', 48.06273858067658),
             ('CHANGE MEMORY', 47.70965074460682),
             ('INFORMATION PROCESSING', 47.70965074460682),
             ('EMIT LASER', 47.274593859658665),
             ('NONAQUEOUS SECONDARY', 47.2